In [14]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from src.data import make_dataset
from src.features import build_features
from src.visualization import visualize

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

HOSPITAL_A_ANLIZAR = 112103  # Este es el Torax

## Obtencion de Métricas de Egresos Hospitalarios

En este apartado se quieren calculcar diversas métricas relevantes para el análisis de los
egresos hospitalarios en Chile. Las métricas a calcular son las siguientes:

1. Cantidad de Egresos Hospitalarios
2. Dias de Estada Totales
3. Cantidad de Intervenciones Quirúrgicas
4. Cantidad de Muertos

Todas estas métricas serán calculadas para cada Año, Establecimiento de Salud y Diagnóstico
distinto que exista en la base de datos. Por ejemplo:

"En el año 2019, el Hospital San José (código XXXXXX) tuvo XX egresos, con XX dias estada promedio,
con XX intervenciones quirúrgicas y XX muertos para el diagnóstico A24.4"s


In [16]:
df_nacional = pl.scan_csv(
    "../data/processed/egresos_procesados.csv",
    infer_schema_length=10000,
    null_values="Extranjero",
)

In [17]:
AGRUPACION = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

metricas = build_features.obtener_metricas_egresos(df_nacional, AGRUPACION).collect()

In [18]:
display(metricas.sample(10))

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,DIAG1,n_egresos,dias_estada_totales,n_int_q,n_muertos
i64,i64,str,str,u32,i64,i64,i64
2007,118202,"""Clínica de la …","""P923""",6,13,0,0
2011,107100,"""Hospital Dr. G…","""L538""",1,5,0,0
2020,121119,"""Hospital de Go…","""H812""",1,10,0,0
2018,114103,"""Hospital Padre…","""S625""",1,1,1,0
2009,102200,"""Clínica Iquiqu…","""N813""",13,32,12,0
2018,114103,"""Hospital Padre…","""N368""",1,1,1,0
2015,110150,"""Hospital San J…","""J159""",6,33,0,0
2004,102100,"""Hospital Dr. E…","""T07X""",5,8,4,0
2016,112101,"""Hospital Dr. L…","""O318""",1,3,0,0


- Al ver una muestra del calculo realizado, se puede observar la cantidad de egresos, dias de
  estada, numero de intervenciones quirurgicas y cantidad de muertos para 10 duplas de
  hospitales - diagnosticos distintos.


Si bien se tienen las metricas calculadas, ahora se quiere responder a las siguientes preguntas:

- En qué diagnósticos se destaca cada hospital productivamente? Esto significa, en qué diagnósticos
  cada hospital tiene una mayor cantidad de egresos que otros hospitales?

- El hospital tiene un desempeño similar dentro de distintos grupos/estratos de hospitales?
  El hospital se desempeñara de la misma forma nacionalmente, dentro del estrato público, privado,
  etc...?

Para responder a la pregunta se realizará un ranking de todos los hospitales para cada
diagnóstico y cada año observado. Este ranking se realizará para distintos grupos/estratos de
hospitales (Nacional, Hospitales Públicos, Hospitales Privados, Hospitales GRD)


## Análisis de Estratos


In [19]:
dict_estratos = build_features.obtener_diccionario_estratos(df_nacional, HOSPITAL_A_ANLIZAR)

In [20]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]

rankings_nacionales = build_features.agregar_ranking_estratos(
    metricas, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

rankings_hospital_analizado = rankings_nacionales.filter(
    pl.col("ESTABLECIMIENTO_SALUD") == HOSPITAL_A_ANLIZAR
)

In [21]:
diags_mas_relevantes = rankings_hospital_analizado.filter(
    (pl.col("ranking_nacionales_n_egresos") == 1)
    & (pl.col("n_int_q") > 0)
    & (pl.col("ANO_EGRESO") > 2011)
).sort(by=["ANO_EGRESO", "n_egresos", "n_int_q"], descending=True)

veinte_mas_relevantes_por_anio = diags_mas_relevantes.groupby(["ANO_EGRESO"]).head(20)

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_16792\516069765.py:7: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  veinte_mas_relevantes_por_anio = diags_mas_relevantes.groupby(["ANO_EGRESO"]).head(20)


In [22]:
DIAGS_MAS_RELEVANTES = veinte_mas_relevantes_por_anio.select(pl.col("DIAG1").unique()).sort(
    by="DIAG1"
)

In [23]:
print(DIAGS_MAS_RELEVANTES)

shape: (45, 1)
┌───────┐
│ DIAG1 │
│ ---   │
│ str   │
╞═══════╡
│ C33X  │
│ C340  │
│ C341  │
│ C342  │
│ …     │
│ Q231  │
│ Q676  │
│ T820  │
│ Z450  │
└───────┘


In [24]:
DIAGS_1_o_2_en_2019 = (
    rankings_hospital_analizado.filter(
        (pl.col("DIAG1").is_in(DIAGS_MAS_RELEVANTES.to_series()))
        & (pl.col("ANO_EGRESO") == 2019)
        & (
            (pl.col("ranking_nacionales_n_egresos") == 1)
            | (pl.col("ranking_nacionales_n_egresos") == 2)
        )
    )
    .select(pl.col("DIAG1"))
)

In [25]:
print(DIAGS_1_o_2_en_2019)

shape: (33, 1)
┌───────┐
│ DIAG1 │
│ ---   │
│ str   │
╞═══════╡
│ Z450  │
│ T820  │
│ M348  │
│ J955  │
│ …     │
│ C342  │
│ C341  │
│ C340  │
│ C33X  │
└───────┘


In [35]:
rankings_hospital_analizado

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,Capítulo,Sección,Categoría,Descripción,DIAG1,n_egresos,dias_estada_totales,n_int_q,n_muertos,ranking_nacionales_n_egresos,total_nacionales_n_egresos,%_nacionales_n_egresos,ranking_publicos_n_egresos,total_publicos_n_egresos,%_publicos_n_egresos,ranking_privados_n_egresos,total_privados_n_egresos,%_privados_n_egresos,ranking_grd_n_egresos,total_grd_n_egresos,%_grd_n_egresos,ranking_interno_n_egresos,total_interno_n_egresos,%_interno_n_egresos
i64,i64,str,str,str,str,str,str,u32,i64,i64,i64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64
2020,112103,"""Instituto Naci…","""Cap.21 FACTOR…","""Z40-Z54 CONTA…","""Z54 CONVALECEN…","""Convalecencia …","""Z548""",1,11,0,0,21,818,0.001222,16,806,0.001241,6,13,0.076923,13,363,0.002755,253,2848,0.000351
2020,112103,"""Instituto Naci…","""Cap.21 FACTOR…","""Z40-Z54 CONTA…","""Z54 CONVALECEN…","""Convalecencia …","""Z540""",5,22,0,0,29,913,0.005476,27,881,0.005675,3,37,0.135135,14,657,0.00761,97,2848,0.001756
2020,112103,"""Instituto Naci…","""Cap.21 FACTOR…","""Z40-Z54 CONTA…","""Z53 PERSONA EN…","""Procedimiento …","""Z530""",1,6,0,0,23,75,0.013333,7,10,0.1,17,66,0.015152,6,9,0.111111,315,2848,0.000351
2020,112103,"""Instituto Naci…","""Cap.21 FACTOR…","""Z40-Z54 CONTA…","""Z51 OTRA ATENC…","""Otras atencion…","""Z518""",15,24,0,0,20,3407,0.004403,10,661,0.022693,11,2761,0.005433,8,610,0.02459,46,2848,0.005267
2020,112103,"""Instituto Naci…","""Cap.21 FACTOR…","""Z40-Z54 CONTA…","""Z51 OTRA ATENC…","""Atención prepa…","""Z514""",118,131,0,0,2,288,0.409722,1,142,0.830986,2,264,0.44697,1,140,0.842857,4,2848,0.041433
2020,112103,"""Instituto Naci…","""Cap.21 FACTOR…","""Z40-Z54 CONTA…","""Z51 OTRA ATENC…","""Otra quimioter…","""Z512""",6,6,0,0,16,2541,0.002361,7,506,0.011858,10,2041,0.00294,7,506,0.011858,87,2848,0.002107
2020,112103,"""Instituto Naci…","""Cap.21 FACTOR…","""Z40-Z54 CONTA…","""Z51 OTRA ATENC…","""Sesión de quim…","""Z511""",195,251,0,0,13,7799,0.025003,7,4206,0.046362,7,3788,0.051478,7,4206,0.046362,3,2848,0.068469
2020,112103,"""Instituto Naci…","""Cap.21 FACTOR…","""Z40-Z54 CONTA…","""Z47 OTROS CUID…","""Cuidados poste…","""Z470""",3,6,3,0,47,1186,0.00253,21,466,0.006438,27,723,0.004149,17,334,0.008982,125,2848,0.001053
2020,112103,"""Instituto Naci…","""Cap.21 FACTOR…","""Z40-Z54 CONTA…","""Z46 PRUEBA Y A…","""Prueba y ajust…","""Z468""",1,10,0,0,8,29,0.034483,6,23,0.043478,3,7,0.142857,4,7,0.142857,251,2848,0.000351


In [71]:
tabla_resumen_egresos_torax_relevantes = (
    rankings_hospital_analizado.filter(
        ((pl.col("ANO_EGRESO") >= 2016) & (pl.col("ANO_EGRESO") <= 2019))
        & (pl.col("DIAG1").is_in(DIAGS_1_o_2_en_2019.to_series()))
    )
    .to_pandas()
    .pivot(
        index=["DIAG1", "Descripción"],
        columns="ANO_EGRESO",
        values=[
            "ranking_nacionales_n_egresos",
            "n_egresos",
            "total_nacionales_n_egresos",
            "%_nacionales_n_egresos",
        ],
    )
).sort_values(
    by=[("ranking_nacionales_n_egresos", 2019), ("n_egresos", 2019)], ascending=[True, False]
)

In [65]:
resumen_egresos_torax_y_nacional_relevantes = (
    tabla_resumen_egresos_torax_relevantes[("n_egresos")].astype(int).astype(str)
    + " ("
    + tabla_resumen_egresos_torax_relevantes[("total_nacionales_n_egresos")].astype(int).astype(str)
    + "; "
    + round((tabla_resumen_egresos_torax_relevantes[("%_nacionales_n_egresos")] * 100), 1).astype(
        str
    )
    + ")"
)

posiciones_torax_egresos_relevantes = tabla_resumen_egresos_torax_relevantes[
    ("ranking_nacionales_n_egresos")
].astype(int)

resumen_egresos_y_posiciones_relevantes = pd.concat(
    [resumen_egresos_torax_y_nacional_relevantes, posiciones_torax_egresos_relevantes], axis=1
)

display(resumen_egresos_y_posiciones_relevantes)

,ANO_EGRESO,2016,2017,2018,2019,2016,2017,2018,2019
DIAG1,Descripción,,,,,,,,
C33X,Tumor maligno de la tráquea,4 (20; 20.0),1 (25; 4.0),1 (18; 5.6),4 (17; 23.5),1,12,11,1
C340,Tumor maligno del bronquio principal,21 (144; 14.6),21 (163; 12.9),17 (143; 11.9),24 (148; 16.2),2,2,2,2
C341,"Tumor maligno del lóbulo superior, bronquio o pulmón",180 (400; 45.0),180 (372; 48.4),220 (478; 46.0),220 (493; 44.6),1,1,1,1
C342,"Tumor maligno del lóbulo medio, bronquio o pulmón",25 (53; 47.2),19 (50; 38.0),24 (68; 35.3),20 (52; 38.5),1,1,1,1
C343,"Tumor maligno del lóbulo inferior, bronquio o pulmón",120 (241; 49.8),118 (258; 45.7),124 (293; 42.3),120 (279; 43.0),1,1,1,1
C381,Tumor maligno del mediastino anterior,9 (26; 34.6),15 (41; 36.6),21 (41; 51.2),19 (52; 36.5),1,1,1,1
C384,Tumor maligno de la pleura,17 (55; 30.9),9 (39; 23.1),9 (44; 20.5),10 (52; 19.2),1,1,1,1
C450,Mesotelioma de la pleura,22 (65; 33.8),23 (81; 28.4),27 (76; 35.5),8 (75; 10.7),1,1,1,2
C780,Tumor maligno secundario del pulmón,43 (328; 13.1),57 (373; 15.3),63 (379; 16.6),62 (430; 14.4),1,1,1,1


,ANO_EGRESO,2016,2017,2018,2019,2016,2017,2018,2019
DIAG1,Descripción,,,,,,,,
C33X,Tumor maligno de la tráquea,4 (20; 20.0),1 (25; 4.0),1 (18; 5.6),4 (17; 23.5),1,12,11,1
C340,Tumor maligno del bronquio principal,21 (144; 14.6),21 (163; 12.9),17 (143; 11.9),24 (148; 16.2),2,2,2,2
C341,"Tumor maligno del lóbulo superior, bronquio o pulmón",180 (400; 45.0),180 (372; 48.4),220 (478; 46.0),220 (493; 44.6),1,1,1,1
C342,"Tumor maligno del lóbulo medio, bronquio o pulmón",25 (53; 47.2),19 (50; 38.0),24 (68; 35.3),20 (52; 38.5),1,1,1,1
C343,"Tumor maligno del lóbulo inferior, bronquio o pulmón",120 (241; 49.8),118 (258; 45.7),124 (293; 42.3),120 (279; 43.0),1,1,1,1
C381,Tumor maligno del mediastino anterior,9 (26; 34.6),15 (41; 36.6),21 (41; 51.2),19 (52; 36.5),1,1,1,1
C384,Tumor maligno de la pleura,17 (55; 30.9),9 (39; 23.1),9 (44; 20.5),10 (52; 19.2),1,1,1,1
C450,Mesotelioma de la pleura,22 (65; 33.8),23 (81; 28.4),27 (76; 35.5),8 (75; 10.7),1,1,1,2
C780,Tumor maligno secundario del pulmón,43 (328; 13.1),57 (373; 15.3),63 (379; 16.6),62 (430; 14.4),1,1,1,1


In [ ]:
visualize.add_dataframes_to_powerpoint(
    [resumen_torax_y_nacional_con_egresos],
    "../reports/presentacion.pptx",
    font_size=9,
    cell_width=12,
    cell_height=0.5,
    font_family="Open Sans",
    max_cell_characters=100,
)

Para hacer un contorl de lo que se calculo, se observará una muestra de 10 hospitales - diagnósticos
distintos


In [ ]:
display(rankings_nacionales.sample(10))

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,Capítulo,Sección,Categoría,Descripción,DIAG1,n_egresos,dias_estada_totales,n_int_q,n_muertos,ranking_nacionales_n_egresos,total_nacionales_n_egresos,%_nacionales_n_egresos,ranking_publicos_n_egresos,total_publicos_n_egresos,%_publicos_n_egresos,ranking_privados_n_egresos,total_privados_n_egresos,%_privados_n_egresos,ranking_grd_n_egresos,total_grd_n_egresos,%_grd_n_egresos,ranking_interno_n_egresos,total_interno_n_egresos,%_interno_n_egresos
i64,i64,str,str,str,str,str,str,u32,i64,i64,i64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64
2008,117101,"""Hospital Clíni…","""Cap.18 SÍNTOM…","""R50-R69 SÍNTO…","""R59 ADENOMEGAL…","""Adenomegalia g…","""R591""",1,8,0,0,13,26,0.038462,8,14,0.071429,null,null,null,7,10,0.1,null,null,null
2017,112528,"""Clínica Las Li…","""Cap.17 MALFOR…","""Q38-Q45 OTRAS…","""Q41 AUSENCIA, …","""Ausencia, atre…","""Q412""",1,11,1,0,6,13,0.076923,null,null,null,3,6,0.166667,null,null,null,null,null,null
2005,121119,"""Hospital de Go…","""Cap.04 ENFERM…","""E08-E14 DIABE…","""E14 DIABETES M…","""Diabetes melli…","""E146""",8,22,0,0,16,616,0.012987,15,563,0.01421,null,null,null,null,null,null,null,null,null
2017,114105,"""Hospital Clíni…","""Cap.10 ENFERM…","""J00-J06 INFEC…","""J04 LARINGITIS…","""Traqueítis agu…","""J041""",1,2,0,0,15,59,0.016949,11,41,0.02439,null,null,null,10,34,0.029412,null,null,null
2005,122201,"""Hospital Padre…","""Cap.09 ENFERM…","""I80-I89 ENFER…","""I88 LINFADENIT…","""Linfadenitis m…","""I880""",14,31,0,0,13,688,0.020349,7,408,0.034314,null,null,null,null,null,null,null,null,null
2018,107217,"""Hospital Naval…","""Cap.11 ENFERM…","""K00-K14 ENFER…","""K11 ENFERMEDAD…","""Sialolitiasis""","""K115""",2,2,2,0,20,108,0.018519,null,null,null,14,66,0.030303,null,null,null,null,null,null
2018,121202,"""Clínica Aleman…","""Cap.21 FACTOR…","""Z40-Z54 CONTA…","""Z47 OTROS CUID…","""Otros cuidados…","""Z478""",1,2,1,0,34,202,0.00495,null,null,null,14,122,0.008197,null,null,null,null,null,null
2019,110270,"""Clínica Los Ma…","""Cap.09 ENFERM…","""I60-I69 ENFER…","""I67 OTRAS ENFE…","""Enfermedad cer…","""I679""",1,8,0,0,124,1199,0.000834,null,null,null,38,299,0.003344,null,null,null,null,null,null
2019,109201,"""Clínica Dávila…","""Cap.13 ENFERM…","""M20-M25 OTROS…","""M21 OTRAS DEFO…","""Otras deformid…","""M215""",4,5,4,0,20,323,0.012384,null,null,null,11,206,0.019417,null,null,null,null,null,null


En la muestra anterior se puede observar las métricas previamente calculada, y el ranking
respectivo que tiene cada hospital para diagnósticos en cada año específico.


In [ ]:
rankings_nacionales.to_pandas().to_csv(
    "../data/interim/ranking_nacional_egresos.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)

rankings_hospital_analizado.to_pandas().to_csv(
    "../data/interim/ranking_torax_egresos.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)

## Metricas para personas sobre 14 años

En este análisis se quiere ver el ranking de egresos pero solamente tomando en cuenta a personas
mayores a 14 años. Esto, para dilucidar si el hospital del tórax es altamente relevante para tratar
patologías congénitas de adultos.


In [ ]:
df_nacional_mayores_a_14 = df_nacional.filter(pl.col("EDAD_A_OS") > 14)

In [ ]:
metricas_sobre_14 = build_features.obtener_metricas_egresos(
    df_nacional_mayores_a_14, AGRUPACION
).collect()

In [ ]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]

rankings_nacionales_sobre_14 = build_features.agregar_ranking_estratos(
    metricas_sobre_14, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

diags_congenitos_relevantes = rankings_nacionales_sobre_14.filter(
    (pl.col("DIAG1").is_in(["Q230", "Q211", "Q231"])) & (pl.col("ANO_EGRESO") == 2019)
)

In [ ]:
diags_congenitos_relevantes.to_pandas().to_csv(
    "../data/interim/diags_congenitas_sobre_14_egresos.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)

## 3. Análisis de procedimientos DEIS

En este análisis se quieren responder las siguientes preguntas:

1. ¿Cuál es el procedimiento más frecuente dentro de los pacientes que se les realizó un procedimiento
   quirúrgico?


In [ ]:
proced = (
    df_nacional.filter(pl.col("INTERV_Q") == 1)
    .groupby(["ANO_EGRESO", "ESTABLECIMIENTO_SALUD", "GLOSA_PROCED_PPAL"])
    .agg(pl.count())
).collect()

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_19856\3397463805.py:3: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  .groupby(["ANO_EGRESO", "ESTABLECIMIENTO_SALUD", "GLOSA_PROCED_PPAL"])


In [ ]:
display(proced)

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_PROCED_PPAL,count
i64,i64,str,u32
2001,103203,null,5921
2001,107212,null,1504
2001,106100,null,9333
2001,107100,null,9959
2001,121111,null,309
2001,105103,null,262
2001,117101,null,1079
2001,112239,null,5848
2001,103210,null,1699


Los resultados muestran el desglose por año y recinto hospitalario junto a la cantidad de
procedimientos realizados en cada intervención quirúrgica. Sin embargo, se puede observar que
existe una cantidad importante de casos donde la glosa del procedimiento fue nula.

Ahora, se quiere calcular cuántas intervenciones quirúrgicas carecen de una glosa de procedimiento.


In [ ]:
display(proced.groupby("GLOSA_PROCED_PPAL").agg(pl.col("count").sum()).sort("count"))

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_19856\989951252.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  display(proced.groupby("GLOSA_PROCED_PPAL").agg(pl.col("count").sum()).sort("count"))


GLOSA_PROCED_PPAL,count
str,u32
"""Enema baritado…",1
"""tonometria ocu…",1
"""Histerosalping…",1
"""Cócleovestibul…",1
"""Criocoagulacio…",1
"""Radiografía Pa…",1
"""Resonancia Mag…",1
"""Tratamiento po…",1
"""Electrocardiog…",1


Por lo tanto, 12382642 de todas las intervenciones quirúrgicas carecen de una glosa de procedimiento.


In [ ]:
proced.write_excel("../data/interim/conteo_procedimientos.xlsx")